#### Базы данных. Паттерн проектирования Singleton

В разработке программного обеспечения паттерн – это общее, многократно используемое решение проблемы, которая часто встречается внутри конкретной ситуации. Это похоже на готовые чертежи, которые можно использовать для решения проблемы в вашем коде.
Стоит отметить, что нельзя применять паттерн проектирования так же, как используется функция из импортированной библиотеки. Вместо этого, вы должны следовать концепции паттерна и реализовать решение, которое соответствует требованиям вашей программы. Паттерн – это не фрагмент кода, а общая концепция, которая описывает, как решить конкретную повторяющуюся проблему.

#### Классификация паттернов проектирования
Изначально существовало две основные классификации паттернов проектирования:

1. Какую проблему решает паттерн.
2. Как относится паттерн к классам или объектам.

Принимая во внимание первую классификацию, паттерны можно разделить на три группы:

1. Порождающие – предоставляют возможность создания контролируемым образом, инициализации и конфигурации объектов, классов и типов данных на основе требуемых критериев.
2. Структурные – помогают организовать структуры связанных объектов и классов, предоставляя новые функциональные возможности.
3. Поведенческие – направлены на выявление общих моделей взаимодействия между объектами.

Позже появились новые паттерны проектирования, из которых можно выделить еще одну категорию:

Concurrency (параллелизм) – это тот тип паттернов проектирования, который имеет дело с многопоточной парадигмой программирования.

#### Паттерн 1: Синглтон
Синглтон (одиночка) – это паттерн проектирования, цель которого ограничить возможность создания объектов данного класса одним экземпляром. Он обеспечивает глобальность до одного экземпляра и глобальный доступ к созданному объекту.

Примеры использования

* Класс в вашей программе имеет только один экземпляр, доступный всем клиентам. Например, один объект базы данных, разделяемый различными частями программы.
* В случае если вам необходим более строгий контроль над глобальными переменными.

In [ ]:
class DataBase:
    __instance = None

    def __new__(cls, *args, **kwargs):
        if cls.__instance is None:
            cls.__instance = super().__new__(cls)
        return cls.__instance

    def __del__(self):
        DataBase.__instance = None

    def __init__(self, dbname, user, password, host):
            self.dbname = dbname
            self.user = user
            self.password = password
            self.host = host

db_users1 = DataBase('users1', 'mysql', 'qwerty', 'localhost')
db_users2 = DataBase('users2', 'postgres', '1234', 'localhost')
print(db_users1)
print(db_users2)

Подведем итоги. Особенности использования Синглтона:

1. Класс имеет только один экземпляр;
2. Вы получаете глобальную точку доступа к этому экземпляру;
3. Синглтон инициализируется только при первом запросе;

In [ ]:
import psycopg2

class Users:
    __connect = None

    def __new__(cls, *args, **kwargs):
        if cls.__connect is None:
            cls.__connect = psycopg2.connect(
            dbname=args[0],
            user=args[1],
            password=args[2],
            host=args[3])
            super().__new__(cls)

        return super().__new__(cls)

    def __del__(self):
        Users.__connect.commit()
        Users.__connect = None

    def __init__(self, dbname, user, password, host):
            self.dbname = dbname
            self.user = user
            self.password = password
            self.host = host

    def select_users(self):
        """Получаем список всех пользователей"""
        with Users.__connect.cursor() as curs:
            curs.execute('SELECT * FROM main.users')
            return curs.fetchall()

    def create_user(self, first_name=None, login=None, password=None):
        # Добавляем нового пользователя
        with Users.__connect.cursor() as curs:
            return curs.execute('INSERT INTO main.users (first_name, login, password) VALUES (%s, %s, %s)', (first_name, login, password))

    def change_password(self, login=None, password=None):
        """Добавляем нового пользователя"""
        with Users.__connect.cursor() as curs:
            return curs.execute('UPDATE main.users SET password=%s WHERE login=%s', (password, login))

db_users = Users('users', 'postgres', '1234', 'localhost')
db_users2 = Users('users', 'postgres', '1234', 'localhost')
select = db_users.select_users()
print(db_users, db_users2)